In [ ]:
import socket
import struct
import time
%matplotlib inline
import scipy, matplotlib.pyplot as plt, IPython.display as ipd
import librosa, librosa.display
#import stanford_mir; stanford_mir.init()
from ipywidgets import interact
import sys
import numpy as np
from IPython.display import Audio, display
import ruptures as rpt  # our package
import pyaudio
import pylab
import time
import sys
import wave
from scipy import signal
import time

#######INFORMATION TO SEND#################

#Function/Declarations
RATE = 44100
CHUNK = int(RATE/20) # RATE / number of updates per second
frames = []
def soundplot(stream):

    t1=time.time()
    #use np.frombuffer if you face error at this line
    #data = np.fromstring(stream.read(CHUNK),dtype=np.int16)
    data = stream.read(CHUNK)
    frames.append(data) 

def find_offset(within_file, find_file, window):
    y_within, sr_within = librosa.load(within_file, sr=None)
    y_find, _ = librosa.load(find_file, sr=sr_within)

    c = signal.correlate(y_within, y_find[:sr_within*window], mode='valid', method='fft')
    peak = np.argmax(c)
    offset = round(peak / sr_within, 2)

    return offset

#BPM/FREQUENCY
duration = 120
x, sr = librosa.load('stromae.wav', duration = duration)
ipd.Audio(x, rate=sr) #Loading Audio File
tempo, beat_times = librosa.beat.beat_track(y=x, sr=sr, start_bpm=60, units='time')
print("BPM:")
print(tempo)
freq = tempo*(1/60)
print("Frequency (Hz):")
print(freq)

#DANCE ROUTINE
hop_length_tempo = 256
oenv = librosa.onset.onset_strength(y=x, sr=sr, hop_length=hop_length_tempo)
tempogram = librosa.feature.tempogram(onset_envelope=oenv,sr=sr,hop_length=hop_length_tempo)
algo = rpt.KernelCPD(kernel="linear").fit(tempogram.T)
n_bkps = 6
bkps = algo.predict(n_bkps=n_bkps)
bkps_times = librosa.frames_to_time(bkps, sr=sr, hop_length=hop_length_tempo)
print("Breakpoints:")
print(bkps_times) #this corresponds to the changes
switch_times = bkps_times
############LAUNCHING THE SERVER############

localIP     = ""

localPort   = 3333

bufferSize  = 1024

# Creating initial packet to send to the mouse
routine_pack = struct.pack("ffffffff",switch_times[0],switch_times[1],
                           switch_times[2],switch_times[3]
                           ,switch_times[4],switch_times[5],switch_times[6],freq)

# Create a datagram socket
UDPServerSocket = socket.socket(family=socket.AF_INET, type=socket.SOCK_DGRAM)

# Bind to address and ip
UDPServerSocket.bind((localIP, localPort))

print("UDP server up and ready to send a packet")

#Initialize connection from the mouse
bytesAddressPair = UDPServerSocket.recvfrom(bufferSize)
message = bytesAddressPair[0]
address = bytesAddressPair[1]
clientMsg = "Message from Client:{}".format(message)
clientIP  = "Client IP Address:{}".format(address)
print(clientMsg)

#Send Dance Routine
UDPServerSocket.sendto(routine_pack, address)

#Recieve confirmation times were sent
time.sleep(2)
bytesAddressPair = UDPServerSocket.recvfrom(bufferSize)
message = bytesAddressPair[0]
address = bytesAddressPair[1]
clientMsg = "Message from Client:{}".format(message)
print(clientMsg)
print("Starting in 3")
time.sleep(1)
print("Starting in 2")
time.sleep(1)
print("Starting in 1")
time.sleep(1)
print("Start!")

#Sending the live audio time in seconds every ~5 seconds
while True:
    start = time.time()
    p=pyaudio.PyAudio()
    stream=p.open(format=pyaudio.paInt16,channels=1,rate=RATE,input=True,
                  frames_per_buffer=CHUNK)
    for i in range(60):    #100 = 5 seconds
        soundplot(stream)
    stream.stop_stream()
    stream.close()
    p.terminate()
    
    wf = wave.open("output.wav",'wb')
    wf.setnchannels(1)
    #wf.setsampwidth(p.get_sample_size(pyaudio.paInt16))
    wf.setsampwidth(2)
    wf.setframerate(RATE)
    wf.writeframes(b''.join(frames))
    wf.close()
    frames = []
    #Finding Offset
    offset = find_offset('stromae.wav', 'output.wav', 10)
    end = time.time()
    delay = end - start
    offset = offset + delay
    print(f"Offset: {offset}s" )
    #Creating Packet to Send to Mouse
    offset_pack = struct.pack("f",offset)
    UDPServerSocket.sendto(offset_pack, address)
    
    #Ping to keep port open for sending packets
    bytesAddressPair = UDPServerSocket.recvfrom(bufferSize)
    message = bytesAddressPair[0]
    
exit

C:\Users\Sammy\anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


BPM:
60.09265988372093
Frequency (Hz):
1.0015443313953487
Breakpoints:
[  7.90639456  15.63863946  48.55292517  80.50358277  88.30548753
 104.02539683 120.00072562]
UDP server up and ready to send a packet
Message from Client:b'Connected to server'
Message from Client:b'Times Recieved'
Starting in 3
Starting in 2
Starting in 1
Start!
Offset: 5.871104898452759s
Offset: 11.001721868515014s
Offset: 16.345123386383058s
Offset: 21.53584442138672s
Offset: 26.778767776489257s
Offset: 32.23679381370545s
Offset: 37.29714182853699s
Offset: 42.31837579727173s
Offset: 47.46663677215576s
Offset: 52.55186937332153s
Offset: 57.67629246711731s
Offset: 62.85249583244324s
Offset: 68.0525220489502s
Offset: 73.21945285797119s
Offset: 78.52359964370727s
Offset: 83.65044024467468s
Offset: 88.62490233421326s
Offset: 93.89679069519043s
Offset: 99.15006339073182s
Offset: 104.5362834262848s
Offset: 109.56969831466675s
Offset: 114.71890200614929s
Offset: 119.89442223548889s
Offset: 124.898565158844s
Offset: 159.

In [ ]:
import socket
import struct
import time
%matplotlib inline
import scipy, matplotlib.pyplot as plt, IPython.display as ipd
import librosa, librosa.display
#import stanford_mir; stanford_mir.init()
from ipywidgets import interact
import sys
import numpy as np
from IPython.display import Audio, display
import ruptures as rpt  # our package
import pyaudio
import pylab
import time
import sys
import wave
from scipy import signal
import time

#######INFORMATION TO SEND#################

#Function/Declarations
RATE = 44100
CHUNK = int(RATE/20) # RATE / number of updates per second
frames = []
def soundplot(stream):

    t1=time.time()
    #use np.frombuffer if you face error at this line
    #data = np.fromstring(stream.read(CHUNK),dtype=np.int16)
    data = stream.read(CHUNK)
    frames.append(data) 

def find_offset(within_file, find_file, window):
    y_within, sr_within = librosa.load(within_file, sr=None)
    y_find, _ = librosa.load(find_file, sr=sr_within)

    c = signal.correlate(y_within, y_find[:sr_within*window], mode='valid', method='fft')
    peak = np.argmax(c)
    offset = round(peak / sr_within, 2)

    return offset

#BPM/FREQUENCY
duration = 120
x, sr = librosa.load('anti-hero.wav', duration = duration)
ipd.Audio(x, rate=sr) #Loading Audio File
tempo, beat_times = librosa.beat.beat_track(y=x, sr=sr, start_bpm=60, units='time')
print("BPM:")
print(tempo)
freq = tempo*(1/60)
print("Frequency (Hz):")
print(freq)

#DANCE ROUTINE
hop_length_tempo = 256
oenv = librosa.onset.onset_strength(y=x, sr=sr, hop_length=hop_length_tempo)
tempogram = librosa.feature.tempogram(onset_envelope=oenv,sr=sr,hop_length=hop_length_tempo)
algo = rpt.KernelCPD(kernel="linear").fit(tempogram.T)
n_bkps = 7
bkps = algo.predict(n_bkps=n_bkps)
bkps_times = librosa.frames_to_time(bkps, sr=sr, hop_length=hop_length_tempo)
print("Breakpoints:")
print(bkps_times) #this corresponds to the changes
switch_times = bkps_times
############LAUNCHING THE SERVER############

localIP     = ""

localPort   = 3333

bufferSize  = 1024

# Creating initial packet to send to the mouse
routine_pack = struct.pack("fffffffff",switch_times[0],switch_times[1],
                           switch_times[2],switch_times[3]
                           ,switch_times[4],switch_times[5],switch_times[6]
                           ,switch_times[7],freq)

# Create a datagram socket
UDPServerSocket = socket.socket(family=socket.AF_INET, type=socket.SOCK_DGRAM)

# Bind to address and ip
UDPServerSocket.bind((localIP, localPort))

print("UDP server up and ready to send a packet")

#Initialize connection from the mouse
bytesAddressPair = UDPServerSocket.recvfrom(bufferSize)
message = bytesAddressPair[0]
address = bytesAddressPair[1]
clientMsg = "Message from Client:{}".format(message)
clientIP  = "Client IP Address:{}".format(address)
print(clientMsg)

#Send Dance Routine
UDPServerSocket.sendto(routine_pack, address)

#Recieve confirmation times were sent
time.sleep(2)
bytesAddressPair = UDPServerSocket.recvfrom(bufferSize)
message = bytesAddressPair[0]
address = bytesAddressPair[1]
clientMsg = "Message from Client:{}".format(message)
print(clientMsg)
print("Starting in 3")
time.sleep(1)
print("Starting in 2")
time.sleep(1)
print("Starting in 1")
time.sleep(1)
print("Start!")

#Sending the live audio time in seconds every ~5 seconds
while True:
    start = time.time()
    p=pyaudio.PyAudio()
    stream=p.open(format=pyaudio.paInt16,channels=1,rate=RATE,input=True,
                  frames_per_buffer=CHUNK)
    for i in range(60):    #100 = 5 seconds
        soundplot(stream)
    stream.stop_stream()
    stream.close()
    p.terminate()
    
    wf = wave.open("output.wav",'wb')
    wf.setnchannels(1)
    #wf.setsampwidth(p.get_sample_size(pyaudio.paInt16))
    wf.setsampwidth(2)
    wf.setframerate(RATE)
    wf.writeframes(b''.join(frames))
    wf.close()
    frames = []
    #Finding Offset
    offset = find_offset('anti-hero.wav', 'output.wav', 10)
    end = time.time()
    delay = end - start
    offset = offset + delay
    print(f"Offset: {offset}s" )
    #Creating Packet to Send to Mouse
    offset_pack = struct.pack("f",offset)
    UDPServerSocket.sendto(offset_pack, address)
    
    #Ping to keep port open for sending packets
    bytesAddressPair = UDPServerSocket.recvfrom(bufferSize)
    message = bytesAddressPair[0]
    
exit

C:\Users\Sammy\anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


BPM:
64.599609375
Frequency (Hz):
1.07666015625
Breakpoints:
[  1.31192744   6.2461678   51.86176871  70.13587302  80.88671202
  95.03927438 107.24136054 120.00072562]
UDP server up and ready to send a packet
Message from Client:b'Connected to server'
Message from Client:b'Times Recieved'
Starting in 3
Starting in 2
Starting in 1
Start!
Offset: 67.60427081108094s
Offset: 9.823065509796143s
Offset: 14.356309661865234s
Offset: 18.902047023773193s
Offset: 23.731204252243042s
Offset: 28.359758825302123s
Offset: 32.930024194717404s
Offset: 37.748472394943235s
Offset: 40.159466142654416s
Offset: 44.75800976753235s
Offset: 49.43829321861267s
Offset: 54.367762575149534s
Offset: 58.5080657863617s
Offset: 63.30317523002625s
Offset: 65.04207087516784s
Offset: 69.41466032028198s
Offset: 73.92884098052978s
Offset: 78.4948655796051s
Offset: 83.124046125412s
Offset: 87.90659752845764s
Offset: 92.67486915588378s
Offset: 99.20307001113892s
Offset: 103.7749814414978s
Offset: 108.3083250427246s
Offset: 1